In [3]:
! pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 5.1 MB/s eta 0:00:00a 0:00:01


In [5]:
import os
import wandb
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
wandb.login()

In [9]:
! pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 3.7 MB/s eta 0:00:00a 0:00:01


In [13]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Initialize wandb
run = wandb.init(
    project="image-classification-demo",
    name="cnn-cifar10",
    config={
        "learning_rate": 0.001,
        "epochs": 10,
        "batch_size": 64,
        "architecture": "CNN",
        "dataset": "CIFAR-10"
    }
)

# Define CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Data loading
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, 
                                 download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, 
                                download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=wandb.config.batch_size, 
                         shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=wandb.config.batch_size, 
                        shuffle=False, num_workers=2)

# Model setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)

# Watch model with wandb
wandb.watch(model, criterion, log="all", log_freq=100)

# Training loop
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, labels) in enumerate(loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Log batch metrics
        if batch_idx % 100 == 0:
            wandb.log({
                "batch_loss": loss.item(),
                "batch_accuracy": 100. * correct / total
            })
    
    return running_loss / len(loader), 100. * correct / total

# Validation function
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(loader), 100. * correct / total

# Training loop
print("Starting training...")
for epoch in range(wandb.config.epochs):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, test_loader, criterion, device)
    
    # Log epoch metrics
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc
    })
    
    print(f'Epoch {epoch+1}/{wandb.config.epochs}:')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

# Save final metrics
run.summary["final_train_accuracy"] = train_acc
run.summary["final_val_accuracy"] = val_acc

# Log model as artifact
artifact = wandb.Artifact('cnn-model', type='model')
torch.save(model.state_dict(), 'model.pth')
artifact.add_file('model.pth')
run.log_artifact(artifact)

print(f'\nFinal Validation Accuracy: {val_acc:.2f}%')
run.finish()

100%|████████████████████████████████████████| 170M/170M [01:10<00:00, 2.43MB/s]


Starting training...
Epoch 1/10:
  Train Loss: 1.4452, Train Acc: 47.09%
  Val Loss: 1.0802, Val Acc: 61.68%
Epoch 2/10:
  Train Loss: 1.0317, Train Acc: 63.22%
  Val Loss: 0.9318, Val Acc: 67.12%
Epoch 3/10:
  Train Loss: 0.8375, Train Acc: 70.41%
  Val Loss: 0.7946, Val Acc: 71.90%
Epoch 4/10:
  Train Loss: 0.7173, Train Acc: 74.88%
  Val Loss: 0.7478, Val Acc: 74.19%
Epoch 5/10:
  Train Loss: 0.6254, Train Acc: 78.03%
  Val Loss: 0.7196, Val Acc: 75.05%
Epoch 6/10:
  Train Loss: 0.5521, Train Acc: 80.38%
  Val Loss: 0.7475, Val Acc: 74.91%
Epoch 7/10:
  Train Loss: 0.4833, Train Acc: 82.76%
  Val Loss: 0.6813, Val Acc: 77.75%
Epoch 8/10:
  Train Loss: 0.4195, Train Acc: 85.19%
  Val Loss: 0.7054, Val Acc: 77.41%
Epoch 9/10:
  Train Loss: 0.3771, Train Acc: 86.41%
  Val Loss: 0.7372, Val Acc: 76.88%
Epoch 10/10:
  Train Loss: 0.3297, Train Acc: 88.14%
  Val Loss: 0.7337, Val Acc: 76.74%

Final Validation Accuracy: 76.74%


batch_accuracy,▁▃▄▄▄▇▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
batch_loss,█▆▆▅▃▄▄▃▃▅▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▂▂▂▂▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▄▅▆▆▇▇▇██
train_loss,█▅▄▃▃▂▂▂▁▁
val_accuracy,▁▃▅▆▇▇████
val_loss,█▅▃▂▂▂▁▁▂▂
batch_accuracy,88.28236
batch_loss,0.29642
epoch,10
final_train_accuracy,88.142
